# Crawling Emotional Words from Excel Data (Melon Comments)

In [1]:
# 필요한 라이브러리 불러오기
import warnings
warnings.filterwarnings(action='ignore')
from selenium import webdriver as wd
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import time
import random
import re
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

In [2]:
# 불용어 사전 불러오기
with open('stop_words.txt', 'r') as file:
    stop_word = file.readline()
    stop_word = str(stop_word)
    
stop_word = stop_word.replace("\ufeff", '').replace("'", '').replace(",", '').replace('\n', '').replace("’", '').replace("‘", '')
stop_words = stop_word.split()


for i in stop_words:
    print(i)

이런
저런
있는
아니
아니고
그랬지만
있는
그러는것
안된
있는디
이쁘
아니니
있데
아니다라면서
아니냐고
같어
있냐
마라
있나
이랬음
아니다
같은디
없었는데
있었는데
많았는데
많음
야하는
근접함
그렇고
그렇지
없는데
같은데
없자나
있지랑
여전히
짧은
미겟
있으신
있었는데
없으면
어떻게
이래
정확히
아니야
없음
미나우를
같긴
아니었다고
아니자
아니라
매운라면
이런거
저런거
있고
확실히
있어야
없어야
스러운
계신
아니죠
있긴
같으면
아닐까
아니였구나
없냐
아님
없어
노래인줄
이럼
아니였어
같음
있던
있는지가
필요하면
아닌가싶고
같아서
익숙하잖니
안됬는데
같은
없는
없겠지
인해
비슷해서
쓰리
있어도
없어도
어느새
같냐
미십
있는게
있다
있음
아니구나
작히
다름
그래
같아요
노래였지
당함
미나우
같아
있으면
아무렇지
그런거
아니지
없네
빠라고
있는거
있어
야한다
있어서
솔직히
야할
노랠
엄연히
없는데서
없다
없어서
그런
있었지
입니당
이상함이
같다는
그럼
아니라서
있는
아니지만
있냐구요
같이가기로
없던
많고
많음
적음
적고
안되고
되고
있구나
없구나
다르군
노래하나는
노래는
아니네
없나
있나
있는데서
다른건
아니었냐고
길어
가능하니까
아닌가
안되요
있게
없는건
그렇다
맛있
당하고
안될까
가능할듯
안녕하세요
있었음
없소
같진
있었으면
없다고
없긴
같다
빠라
그러게
미안해
어떤
망할수
안된다
웬만하면
왠만하면
어떠십니까
노래였다
가능했는데
아닌데
그럼에도
입니다
아닌가요
없엇다
많은
그러는데요
아니여도
그럴듯한
다르냐
똑같은
이럴
없을
아니던데
있을줄
있나여
낮은
그랬어
같구
있자나여
있었잖아
싫어
공허해
가능한
그러셨어요
그만
그만해
있겠지만
있었구먼
그런지
쎄여
궁금한게
많하
있습니다
길다
아니었구나
그만하라며
없다거나
없겠다
있는데
있었어요
많아
아니냐구요
부탁드립니다
같은데에서
이런데
노래였는데
뿐인줄
노래내고
있었네
아닌대
안녕하세
같은건
있고
있네
딱인데
검은
있으시면
쓰리디
아니면
아니더라도
있잖아요
없다는건
없었을텐데
아닙니다
많은데
그런듯
괜찮았겠다
안됩니

In [3]:
# 곡 제목, 유튜브 링크가 있는 데이터 프레임(엑셀 파일)을 불러옴
data = pd.read_excel('/Users/lifeofpy/WebCrwaling/Melon/melon_with_comments.xlsx')
data

,Song Title,Site Link,Emotional Words
0,미쳐,https://www.melon.com/song/detail.htm?songId=5...,NaN
1,Badboy,https://www.melon.com/song/detail.htm?songId=3...,NaN
2,HIP,https://www.melon.com/song/detail.htm?songId=3...,NaN
3,DUN DUN,https://www.melon.com/song/detail.htm?songId=3...,NaN
4,How You Like That,https://www.melon.com/song/detail.htm?songId=3...,NaN
5,TT,https://www.melon.com/song/detail.htm?songId=3...,NaN
6,뿜뿜,https://www.melon.com/song/detail.htm?songId=3...,NaN
7,롤린,https://www.melon.com/song/detail.htm?songId=3...,NaN
8,Alcohol-Free,https://www.melon.com/song/detail.htm?songId=3...,NaN
9,DunDun Dance,https://www.melon.com/song/detail.htm?songId=3...,NaN


In [4]:
data['Emotional Words'][0]  # 비어있는 nan 값의 Emotional Words 열을 감정 단어로 채우자!

nan

In [5]:
data_lst = data['Site Link'].tolist() # Series to List
data_lst

['https://www.melon.com/song/detail.htm?songId=5584641',
 'https://www.melon.com/song/detail.htm?songId=30864993',
 'https://www.melon.com/song/detail.htm?songId=32175937',
 'https://www.melon.com/song/detail.htm?songId=32351853',
 'https://www.melon.com/song/detail.htm?songId=32720013',
 'https://www.melon.com/song/detail.htm?songId=30054344',
 'https://www.melon.com/song/detail.htm?songId=30816860',
 'https://www.melon.com/song/detail.htm?songId=30287019',
 'https://www.melon.com/song/detail.htm?songId=33559180',
 'https://www.melon.com/song/detail.htm?songId=33464805',
 'https://www.melon.com/song/detail.htm?songId=30568338',
 'https://www.melon.com/song/detail.htm?songId=31492319',
 'https://www.melon.com/song/detail.htm?songId=32872978',
 'https://www.melon.com/song/detail.htm?songId=32393669']

In [6]:
driver = wd.Chrome(executable_path='/Users/lifeofpy/Desktop/chromedriver')

In [7]:
for k in range(len(data)):
    
    # 댓글 페이지 개수 구하기
    driver = wd.Chrome(executable_path='/Users/lifeofpy/Desktop/chromedriver')
    page_url = data_lst[k] # url 에 페이지 링크를 하나씩 담아준다
    driver.get(page_url)
    comments = []
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    pages = soup.find_all('span', {'class': 'd_cmtpgn_srch_cnt'})[0]

    # 일단 추출된 태그를 문자열로 바꾸기
    pages = str(pages)

    # 정규식을 통해 태그에서 숫자(=댓글 개수)만 추출하기
    cmt_num = re.findall('\d+', pages)
    page_num = "".join(cmt_num)
    page_num = int(page_num)
    page_num = page_num//10
        
    # 댓글 페이지마다 바뀔 url 생성
    url = page_url + '#cmtpgn=&pageNo={}&sortType=0&srchType=2&srchWord='
    
    
    for i in range(0, page_num+1):
        link = url.format(i) # {} 안에 들어갈 문자열 포매팅
        driver.get(link)
        driver.implicitly_wait(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        comment = soup.find_all('div', {'class': 'd_cmtpgn_cmt_full_contents'})

        for r in comment:
            comments.append(r.get_text().strip())
    
    
    melon_comments = []
    melon_comments = comments
    
    # 댓글이 모인 comments 리스트에서 필요없는 문자 처리하기
    for j in range(len(melon_comments)):
        if '내용' in melon_comments[j]:
            melon_comments[j] = melon_comments[j].replace('내용', '')
    
    # '감정 단어'만 추출하기
    okts = []
    
    for i in range(len(melon_comments)):
        okts.append(okt.pos(melon_comments[i]))
    
    # '감정 단어'의 품사는 '형용사'이므로 형용사 태그만 adj 리스트에 담기
    adj = []

    for i in range(len(okts)):
        for j in range(len(okts[i])):
            if okts[i][j][1] == 'Adjective' and len(okts[i][j][0]) > 1:
                adj.append(okts[i][j][0])
                
    real_adj = []
    
    # 불용어 제외하고 real_adj 에 담기
    for w in adj:
        if w not in stop_words:
            real_adj.append(w)
            

    data.iloc[k, 2] = str(real_adj)

In [8]:
# 데이터프레임 엑셀로 만들기
data.to_excel('Video_with_Emotional_Words_Melon.xlsx', index=False)

In [9]:
# 크롤링 정리 결과
data

,Song Title,Site Link,Emotional Words
0,미쳐,https://www.melon.com/song/detail.htm?songId=5...,"['미쳐가는', '미친', '미쳐가는', '미친', '미쳐가는', '미친', '찰짐..."
1,Badboy,https://www.melon.com/song/detail.htm?songId=3...,"['미쳐', '좋음', '미치게', '미친', '좋고', '좋았음', '예쁨', '..."
2,HIP,https://www.melon.com/song/detail.htm?songId=3...,"['좋다', '좋다', '미쳤어', '좋다', '좋다', '미쳤어', '좋다', '..."
3,DUN DUN,https://www.melon.com/song/detail.htm?songId=3...,"['좋네', '비슷하긴한데', '좋네', '비슷하긴한데', '좋네', '비슷하긴한데..."
4,How You Like That,https://www.melon.com/song/detail.htm?songId=3...,"['좋음', '좋겠다는', '좋다', '미쳤냐', '좋음', '좋겠다는', '좋다'..."
5,TT,https://www.melon.com/song/detail.htm?songId=3...,"['열광하던', '열광하던', '열광하던', '설렌다', '신기한', '좋으니까',..."
6,뿜뿜,https://www.melon.com/song/detail.htm?songId=3...,"['신나는', '예뻐서', '좋아했던', '설렘', '좋음', '좋다', '좋고',..."
7,롤린,https://www.melon.com/song/detail.htm?songId=3...,"['좋아라', '좋아할거', '싫어요', '좋음', '좋아라', '좋아할거', '싫..."
8,Alcohol-Free,https://www.melon.com/song/detail.htm?songId=3...,"['많은것도', '사랑스러운건', '좋아서', '예쁨', '궁금한', '좋은데', ..."
9,DunDun Dance,https://www.melon.com/song/detail.htm?songId=3...,"['좋아요', '경쾌해', '완벽해요', '사랑스러요', '좋네요', '미쳤어', ..."
